# 2. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)
- LDA는 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정합니다. 데이터가 주어지면, LDA는 문서가 생성되던 과정을 역추적합니다.

## 1) LDA 개요
- Example
    - 문서1 : 저는 사과랑 바나나를 먹어요
    - 문서2 : 우리는 귀여운 강아지가 좋아요
    - 문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요

- LDA를 수행할 때, 문서 집합에서 토픽 개수 k를 사용자가 설정(여기선 k=2로 설정)
- k값 잘못 선택하면 이상한 결과 나올 수 있음(k는 hyperparameter)

### <각 문서의 토픽 분포>
문서1 : 토픽 A 100%  
문서2 : 토픽 B 100%  
문서3 : 토픽 B 60%, 토픽 A 40%  

### <각 토픽의 단어 분포>
**토픽A** : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜찍하고 0%, 좋아요 0%  
**토픽B** : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%

LDA는 토픽의 제목을 정해주지 않지만, 이 시점에서 알고리즘의 사용자는 위 결과로부터 두 토픽이 각각 과일에 대한 토픽과 강아지에 대한 토픽이라고 판단

## 2) LDA의 가정
- 빈도수 기반의 표현 방법인 BoW의 행렬 **DTM** 또는 **TF-IDF** 행렬을 input으로 함(단어 순서 신경X)

(1) 문서에 사용할 단어의 개수 N을 정합니다.
> Ex) 5개의 단어를 정하였습니다.

(2) 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정합니다.
> Ex) 위 예제와 같이 토픽이 2개라고 하였을 때 강아지 토픽을 60%, 과일 토픽을 40%와 같이 선택할 수 있습니다.

(3) 문서에 사용할 각 단어를 (아래와 같이) 정합니다.  
(3)-1 토픽 분포에서 토픽 T를 확률적으로 고릅니다.
> Ex) 60% 확률로 강아지 토픽을 선택하고, 40% 확률로 과일 토픽을 선택할 수 있습니다.

(3)-2 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고릅니다.
> Ex) 강아지 토픽을 선택하였다면, 33% 확률로 강아지란 단어를 선택할 수 있습니다. 이제 3)을 반복하면서 문서를 완성합니다.

**이러한 과정을 통해 문서가 작성되었다는 가정 하에 LDA는 토픽을 뽑아내기 위하여 위 과정을 역으로 추적하는 역공학(reverse engneering)을 수행**

## 3) LDA의 수행하기
### 1) 사용자는 알고리즘에서 토픽의 개수 k를 설정
앞서 말하였듯이 LDA에게 토픽의 개수를 알려주는 역할은 사용자의 역할입니다. LDA는 토픽의 개수 k를 입력받으면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정합니다.

### 2) 각 단어를 하나의 토픽에 할당
이제 LDA는 모든 문서의 모든 단어에 대해서 k개 중 하나의 토픽을 랜덤으로 할당합니다. 이 작업이 끝나면 각 문서는 토픽을 가지며, 토픽은 단어 분포를 가지는 상태입니다. 물론 랜덤으로 할당하였기 때문에 사실 이 결과는 전부 틀린 상태입니다. 만약 한 단어가 한 문서에서 2회 이상 등장하였다면, 각 단어는 서로 다른 토픽에 할당되었을 수도 있습니다.

### 3) 모든 문서의 모든 단어에 대해서 아래의 사항을 반복(iterative)

#### 3-1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당됩니다.
- $P$(topic $t$ | document $d$) : 문서 $d$의 단어들 중 토픽 $t$에 해당하는 단어들의 비율
- $P$(word $w$ | topic $t$) : 단어 $w$를 갖고 있는 모든 문서들 중 토픽 $t$가 할당된 비율

이를 반복하면, 모든 할당이 완료된 수렴 상태가 됩니다. 두 가지 기준이 어떤 의미인지 예를 들어보겠습니다. 설명의 편의를 위해서 두 개의 문서라는 새로운 예를 사용합니다.

<img src=lda1.png width=400>

doc1의 세번째 단어 apple의 토픽을 결정하기 위해,
#### 첫번째 기준 : 문서 doc1의 단어들이 어떤 토픽에 해당되는지 봄
- doc1에서 모든 단어들은 토픽A와 토픽B에 50:50의 비율로 할당되어 있으므로 이 기준에 따르면 apple은 토픽A 또는 토픽 B 둘 중 어디에도 속할 가능성이 있음

<img src=lda3.png width=400>

#### 두번째 기준 : apple이 모든 각 문서마다 어떤 토픽에 할당되어져 있는지 봄
- 이 기준에 따르면 apple은 토픽B에 할당될 가능성이 높음

<img src=lda2.png width=400>

※ 이 두 기준을 참고하여 LDA는 doc1의 apple을 어떤 토픽에 할당할 지 결정

## 4) 잠재 디리클레 할당과 잠재 의미 분석의 차이
- LSA : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.
- LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

## 5) 실습을 통한 이해
- 이번 챕터에서는 **gensim**을 사용할 예정
- 사이킷런으로 LDA 실습 : https://wikidocs.net/40710

### 정수 인코딩과 단어 집합 만들기
- Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 뉴스 데이터를 다시 사용

In [1]:
# 이전 section에서 했던 똑같은 전처리

import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

# 불용어 제거
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

- 각 단어에 **정수 인코딩**을 하면서 동시에 각 뉴스에서의 단어마다 **빈도수** 기록할 예정  
$\Longrightarrow$ (word_id, word_frequency)  
$\Longrightarrow$ **gensim의 corpora.Dictionary()**를 사용하여 손쉽게 구할 수 있음


In [9]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
print(dictionary, '\n')

corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력

Dictionary(64281 unique tokens: ['acts', 'atrocities', 'austria', 'away', 'biased']...) 

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [10]:
# (66,2)는 정수 인코딩이 66으로 할당된 단어가 두번째 뉴스에서 두 번 등장했다는 의미
print(dictionary[66]) # 66으로 인코딩된 단어는 faith

faith


In [11]:
print(len(dictionary)) # 총 65284개 단어가 학습

64281


### LDA 모델 훈련시키기
- 기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있었으므로 토픽의 개수를 20으로 하여 LDA 모델을 학습

In [12]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, 
                                           id2word=dictionary, passes=15) # passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.033*"space" + 0.012*"nasa" + 0.010*"health" + 0.009*"medical"')
(1, '0.022*"period" + 0.014*"play" + 0.013*"myers" + 0.012*"power"')
(2, '0.009*"chip" + 0.008*"system" + 0.008*"would" + 0.007*"encryption"')
(3, '0.025*"henrik" + 0.010*"financial" + 0.009*"planes" + 0.009*"pointer"')
(4, '0.009*"church" + 0.007*"entries" + 0.007*"objective" + 0.004*"must"')
(5, '0.011*"said" + 0.010*"people" + 0.006*"know" + 0.006*"armenian"')
(6, '0.014*"available" + 0.013*"information" + 0.011*"mail" + 0.010*"list"')
(7, '0.027*"water" + 0.026*"ground" + 0.021*"wire" + 0.017*"wiring"')
(8, '0.030*"bike" + 0.013*"koresh" + 0.012*"ride" + 0.010*"riding"')
(9, '0.010*"goal" + 0.010*"puck" + 0.010*"kings" + 0.010*"flames"')
(10, '0.014*"would" + 0.012*"people" + 0.009*"think" + 0.007*"know"')
(11, '0.054*"file" + 0.046*"entry" + 0.044*"output" + 0.022*"section"')
(12, '0.009*"israel" + 0.009*"jews" + 0.008*"state" + 0.008*"government"')
(13, '0.030*"sale" + 0.023*"offer" + 0.021*"please" + 0.020*"s

- 각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줌
- 총 20개의 토픽
- passes는 passes는 알고리즘의 동작 횟수를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 됌
- num_words=4로 총 4개의 단어만 출력하도록 하였음
- 만약 10개의 단어를 출력하고 싶다면 아래의 코드를 수행하면 됌(10은 default)
    - print(ldamodel.print_topics())

### LDA 시각화 하기

In [13]:
!pip install pyLDAvis

  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97715 sha256=02cf52ca5a890ba082a6b653db36a4647cf2aae2b4edb3d6e4898ea95db14564
  Stored in directory: C:\Users\jnh78\AppData\Local\pip\Cache\wheels\98\71\24\513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491063 sha256=dda223881131a45eb77a6a31df97573b7b8ab69cdecdfaa9e3d814d4f5535c41
  Stored in directory: C:\Users\jnh78\AppData\Local\pip\Cache\wheels\8b\99\a0\81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32045 sha256=f18360eadaba64bed6b1ffbbf58aadde81001dcfcb26fb37e80e2b0099a046e8
  Stored in directory: C:\Users\jnh78\AppData\Local\pip\Cache\wheels\20\5a\d8\1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis future funcy


In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\jnh78\Anaconda3\envs\tutorial\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


- 좌측의 원들은 각각의 20개의 토픽을 나타냄
- 두 개의 원의 최단 거리는 각 토픽들이 다름의 정도
    - 만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미


- 위의 그림에서는 10번 토픽을 클릭 $\Longrightarrow$ 우측에는 10번 토픽에 대한 정보가 나타남
- 한 가지 <ins>주의</ins>할 점은 LDA 모델의 출력 결과에서는 토픽 번호가 0부터 할당되어 0\~19의 숫자가 사용된 것과는 달리 위의 LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽 번호는 이제 +1이 된 값인 1~20까지의 값을 가집니다.

### 문서 별 토픽 분포 보기
- 위에서는 토픽 별 다넝 분포 확인 가능 But, 문서 별 토픽 분포는 확인X
- 각 문서의 토픽 분포는 이미 훈련된 LDA 모델인 ldamodel[]에 전체 데이터가 정수 인코딩 된 결과를 넣은 후에 확인이 가능
- 여기서는 지면의 한계로 상위 5개의 문서에 대해서만 토픽 분포 확인

In [15]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(10, 0.19776207), (12, 0.7877079)]
1 번째 문서의 topic 비율은 [(2, 0.033144612), (9, 0.027444731), (10, 0.87701505), (18, 0.04132043)]
2 번째 문서의 topic 비율은 [(2, 0.09290342), (7, 0.033501767), (10, 0.46720037), (12, 0.39324024)]
3 번째 문서의 topic 비율은 [(2, 0.53585297), (4, 0.018690664), (11, 0.03202024), (15, 0.27314106), (17, 0.07995413), (19, 0.049361587)]
4 번째 문서의 topic 비율은 [(11, 0.078518), (18, 0.88813937)]


- (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미
    - 예를 들어, 0번째 문서의 토픽 비율에서 (7, 0.3050222)은 7번 토픽이 30%의 분포도를 가지는 것을 의미

In [16]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [17]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,12.0,0.7877,"[(10, 0.19775085), (12, 0.78771913)]"
1,1,10.0,0.8771,"[(2, 0.03314993), (9, 0.027444726), (10, 0.877..."
2,2,10.0,0.4672,"[(2, 0.09288275), (7, 0.033501796), (10, 0.467..."
3,3,2.0,0.5359,"[(2, 0.53585196), (4, 0.018690664), (11, 0.032..."
4,4,18.0,0.8881,"[(11, 0.078519724), (18, 0.88813764)]"
5,5,10.0,0.7357,"[(9, 0.073214404), (10, 0.7356963), (19, 0.152..."
6,6,2.0,0.2703,"[(2, 0.27026263), (10, 0.16597119), (15, 0.217..."
7,7,10.0,0.4879,"[(2, 0.25914505), (5, 0.086509734), (10, 0.487..."
8,8,10.0,0.8018,"[(10, 0.8017587), (16, 0.1716848)]"
9,9,15.0,0.6779,"[(2, 0.12941782), (4, 0.07858744), (15, 0.6779..."
